In [11]:
import numpy as np
import pandas as pd

In [22]:
data = open('data.txt','r')
schedule_string = data.read()

In [23]:
import re

class CourseScheduleExtractor:
    def __init__(self, schedule_string):
        self.schedule_string = schedule_string

    def extract_course_info(self):
        pattern = r'(\b[A-Z]{3} \d{3}-\d+)\s+(.+)\s+([A-Za-zÇçĞğİıÖöŞşÜü\s]+)\s+header'
        matches = re.findall(pattern, self.schedule_string)
        course_info_list = []
        new_str= schedule_string.split('header=[Hint] body=[Syllabus]')
        #print(new_str)
        for match in matches:
            #weekdays = []
            #hours = []
            lecture_hours = []
            i = matches.index(match)
            #print(i)
            course = match[0].strip().split('-')
            course_code = course[0].split(' ')[0] + '-' + course[0].split(' ')[1]
            section_number = course[1]
            course_name = match[1].strip()
            instructor_name = match[2].strip()
            parts = re.findall(r'(Mon|Tue|Wed|Thu|Fri) (\d{2}:\d{2}-\d{2}:\d{2})', new_str[i])
            for part in parts:
                #weekdays.append(part[0])
                #hours.append(part[1])
                lecture_hours.append(part[0] + ' ' + part[1])
            course_info = {
                "Course Code": course_code,
                "Section Number": section_number,
                "Course Name": course_name,
                "Instructor Name": instructor_name,
                #"Weekdays": weekdays,
                #"Hours": hours,
                "Lecture Hours": lecture_hours
            }
            course_info_list.append(course_info)
        return course_info_list
    
    def get_course_info(self, course_code):
        course_info_list = self.extract_course_info()
        course_codes = list(set(course_info["Course Code"] for course_info in course_info_list))
        course_codes.sort()
        filtered_courses = []
        for course in range(len(course_codes)):
            filtered_courses.append([course_info for course_info in course_info_list if course_info["Course Code"] == str(course_codes[course])])
        return filtered_courses[course_codes.index(course_code)]

In [24]:
# If I want to add EEE-313, EEE-321 and EEE-351 courses to my schedule:

rows=['1', '2', '3', '4', 'L', '5', '6', '7', '8']
cols=['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
df = pd.DataFrame(index=rows, columns=cols, data='-')

for course in ['EEE-313', 'EEE-321', 'EEE-351']:
    course_section_list = CourseScheduleExtractor(schedule_string).get_course_info(course)
    section_num = len(course_section_list)
    print(course + ' has ' + str(section_num) + ' sections')

    for i in range(section_num):
        hours = course_section_list[i]['Lecture Hours']
        
        for j in range(len(hours)):
            
            if hours[j].split(' ')[1] == '08:30-10:20':
                df.loc['1', hours[j].split(' ')[0]] = course + '-' + str(i+1)
                df.loc['2', hours[j].split(' ')[0]] = course + '-' + str(i+1)

EEE-313 has 3 sections
EEE-321 has 2 sections
EEE-351 has 3 sections


In [25]:
df

,Mon,Tue,Wed,Thu,Fri
1,EEE-321-1,-,EEE-351-3,EEE-351-1,-
2,EEE-321-1,-,EEE-351-3,EEE-351-1,-
3,-,-,-,-,-
4,-,-,-,-,-
L,-,-,-,-,-
5,-,-,-,-,-
6,-,-,-,-,-
7,-,-,-,-,-
8,-,-,-,-,-
